In [6]:
 import numpy as np

# Main Gui 

In [3]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import os, cv2
import pickle

# Create the tkinter app
app = tk.Tk()

# Define the function to select the destination folder
def select_folder():
    folder_name = folder_name_entry.get().strip()
    if not folder_name:
        messagebox.showerror('Error', 'Please enter a folder name')
        return
    dest_folder = os.path.join('images', folder_name)
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder, exist_ok=True)
    for i in range(3):
        file_path = filedialog.askopenfilename(title=f'Select Image File {i+1}', filetypes=[('Image Files', '*.jpg;*.jpeg;*.png')])
        if file_path:
            file_name = os.path.basename(file_path)
            dest_file_path = os.path.join(dest_folder, file_name)
            if os.path.exists(dest_file_path):
                overwrite = messagebox.askyesno('Overwrite file', f'{file_name} already exists in {folder_name}. Do you want to overwrite it?')
                if not overwrite:
                    continue
            img = Image.open(file_path)
            img.save(dest_file_path)
            messagebox.showinfo('Success', f'Successfully saved image {i+1} to {dest_file_path}')
        else:
            messagebox.showerror('Error', f'No image file selected for image {i+1}')

# Define the function to exit the app
def exit_app():
    app.destroy()

# Define the function to restart the app
def restart_app():
    file_path = os.path.join(os.getcwd(), "recognizers", "face-trainner.yml")
    try:
        os.remove(file_path)
        messagebox.showinfo("Restart", "Your program restarted successfully")
    except OSError:
        messagebox.showwarning("Restart", "Error. Please try again.")
    app.destroy()
    
    
# Define the function to delete the user
def delete_user():
    folder_name = folder_name_entry.get().strip()
    if not folder_name:
        messagebox.showerror('Error', 'Please enter a folder name')
        return
    dest_folder = os.path.join('images', folder_name)
    if os.path.exists(dest_folder):
        confirm = messagebox.askyesno('Delete user', f'Are you sure you want to delete user {folder_name}? This will permanently delete all images in {dest_folder}.')
        if confirm:
            for file_name in os.listdir(dest_folder):
                file_path = os.path.join(dest_folder, file_name)
                os.remove(file_path)
            os.rmdir(dest_folder)
            messagebox.showinfo('Success', f'Successfully deleted user {folder_name}.')
    else:
        messagebox.showerror('Error', f'User {folder_name} does not exist.')

        

def open_cam():
    face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read('recognizers/face-trainner.yml')

    labels = {"person_name": 1}
    with open("pickles/face-labels.pickle", 'rb') as f:
        og_labels = pickle.load(f)
        labels = {v:k for k,v in og_labels.items()}

    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)

        for (x, y, w, h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]

            id_, conf = recognizer.predict(roi_gray)
            if conf>=4 and conf<=85:
                font = cv2.FONT_HERSHEY_SIMPLEX
                name = labels[id_]
                color = (255, 255, 255)
                stroke = 2
                cv2.putText(frame, name, (x,y), font, 1, color, stroke, cv2.LINE_AA)

            img_item = "7.png"
            cv2.imshow('roi', roi_gray)

            color = (255, 0, 0)
            thickness = 2
            end_cord_x = x + w
            end_cord_y = y + h
            cv2.rectangle(frame, (x, y), (end_cord_x, end_cord_y), color, thickness)

        cv2.imshow('frame', frame)

        if cv2.waitKey(20) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Set up the tkinter app window
app.title('Select Images')
app.geometry('400x250')
app.resizable(False, False)
screen_width = app.winfo_screenwidth()
screen_height = app.winfo_screenheight()
x = int((screen_width - app.winfo_reqwidth()) / 2)
y = int((screen_height - app.winfo_reqheight()) / 2)
app.geometry(f'+{x}+{y}')
label = tk.Label(app, text='Enter the name of the user:')
label.pack()
folder_name_entry = tk.Entry(app)
folder_name_entry.pack()
button_frame = tk.Frame(app)
button_frame.pack()
select_button = tk.Button(button_frame, text='Select', command=select_folder)
select_button.pack(side=tk.LEFT, padx=5)
restart_button = tk.Button(button_frame, text='Restart', command=restart_app)
restart_button.pack(side=tk.LEFT, padx=5)
delete_button = tk.Button(button_frame, text='Delete User', command=delete_user)
delete_button.pack(side=tk.LEFT, padx=5)
exit_button = tk.Button(button_frame, text='Exit', command=exit_app)
exit_button.pack(side=tk.LEFT, padx=5)
open_button = tk.Button(button_frame, text='Open Camera', command=open_cam)
open_button.pack(side=tk.LEFT, padx=5) 
app.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "D:\New folder\envs\python\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\Hp\AppData\Local\Temp\ipykernel_11632\2127327312.py", line 72, in open_cam
    recognizer.read('recognizers/face-trainner.yml')
cv2.error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv_contrib\modules\face\src\facerec.cpp:61: error: (-2:Unspecified error) File can't be opened for reading! in function 'cv::face::FaceRecognizer::read'



# Train

In [ ]:
import os
from PIL import Image
import cv2
import numpy as np
import pickle



BASE_DIR = os.getcwd()
image_dir = os.path.join(BASE_DIR, "images")


face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()

current_id = 0
label_ids = {}
y_labels = []
x_train = []

for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith("png") or file.endswith("jpg"):
            path = os.path.join(root, file)
            label = os.path.basename(root).replace(" ", "-").lower()
            print(label, path)
            if not label in label_ids:
                label_ids[label]= current_id
                current_id +=1
            id_= label_ids[label]
            print(label_ids)
            
            pil_image = Image.open(path).convert("L")
            size = (550,550)
            final_image = pil_image.resize(size,Image.Resampling.LANCZOS)
            image_array = np.array(final_image, "uint8")
            #print(image_array)
            faces = face_cascade.detectMultiScale(image_array, scaleFactor=1.5, minNeighbors=5)
            
            
            for(x,y,w,h) in faces:
                roi= image_array[y:y+h, x:x+w]
                x_train.append(roi)
                y_labels.append(id_)

            
            
with open("pickles/face-labels.pickle", 'wb') as f:
	pickle.dump(label_ids, f)

recognizer.train(x_train, np.array(y_labels))
recognizer.save("recognizers/face-trainner.yml")

In [ ]:
import numpy as np
import cv2
import pickle

face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('recognizers/face-trainner.yml')

labels = {"person_name": 1}
with open("pickles/face-labels.pickle", 'rb') as f:
	og_labels = pickle.load(f)
	labels = {v:k for k,v in og_labels.items()}

cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        id_, conf = recognizer.predict(roi_gray)
        if conf>=4 and conf<=85:
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id_]
            color = (255, 255, 255)
            stroke = 2
            cv2.putText(frame, name, (x,y), font, 1, color, stroke, cv2.LINE_AA)

        
        img_item = "7.png"
        cv2.imshow('roi', roi_gray)

        color = (255, 0, 0)
        thickness = 2
        end_cord_x = x + w
        end_cord_y = y + h
        cv2.rectangle(frame, (x, y), (end_cord_x, end_cord_y), color, thickness)

    cv2.imshow('frame', frame)

    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows() 


In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import os, cv2
import pickle

# Create the tkinter app
app = tk.Tk()

# Define the function to select the destination folder
def select_folder():
    folder_name = folder_name_entry.get().strip()
    if not folder_name:
        messagebox.showerror('Error', 'Please enter a folder name')
        return
    dest_folder = os.path.join('images', folder_name)
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder, exist_ok=True)
    for i in range(3):
        file_path = filedialog.askopenfilename(title=f'Select Image File {i+1}', filetypes=[('Image Files', '*.jpg;*.jpeg;*.png')])
        if file_path:
            file_name = os.path.basename(file_path)
            dest_file_path = os.path.join(dest_folder, file_name)
            if os.path.exists(dest_file_path):
                overwrite = messagebox.askyesno('Overwrite file', f'{file_name} already exists in {folder_name}. Do you want to overwrite it?')
                if not overwrite:
                    continue
            img = Image.open(file_path)
            img.save(dest_file_path)
            messagebox.showinfo('Success', f'Successfully saved image {i+1} to {dest_file_path}')
        else:
            messagebox.showerror('Error', f'No image file selected for image {i+1}')

# Define the function to exit the app
def exit_app():
    app.destroy()

# Define the function to restart the app
def restart_app():
    file_path = os.path.join(os.getcwd(), "recognizers", "face-trainner.yml")
    try:
        os.remove(file_path)
        messagebox.showinfo("Restart", "Your program restarted successfully")
    except OSError:
        messagebox.showwarning("Restart", "Error. Please try again.")
    app.destroy()


# Define the function to delete the user
def delete_user():
    folder_name = folder_name_entry.get().strip()
    if not folder_name:
        messagebox.showerror('Error', 'Please enter a folder name')
        return
    dest_folder = os.path.join('images', folder_name)
    if os.path.exists(dest_folder):
        confirm = messagebox.askyesno('Delete user', f'Are you sure you want to delete user {folder_name}? This will permanently delete all images in {dest_folder}.')
        if confirm:
            for file_name in os.listdir(dest_folder):
                file_path = os.path.join(dest_folder, file_name)
                os.remove(file_path)
            os.rmdir(dest_folder)
            messagebox.showinfo('Success', f'Successfully deleted user {folder_name}.')
    else:
        messagebox.showerror('Error', f'User {folder_name} does not exist.')

        

def open_cam():
    face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read('recognizers/face-trainner.yml')

    labels = {"person_name": 1}
    with open("pickles/face-labels.pickle", 'rb') as f:
        og_labels = pickle.load(f)
        labels = {v:k for k,v in og_labels.items()}

    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)

        for (x, y, w, h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]

            id_, conf = recognizer.predict(roi_gray)
            if conf>=4 and conf<=85:
                font = cv2.FONT_HERSHEY_SIMPLEX
                name = labels[id_]
                color = (255, 255, 255)
                stroke = 2
                cv2.putText(frame, name, (x,y), font, 1, color, stroke, cv2.LINE_AA)

            img_item = "7.png"
            cv2.imshow('roi', roi_gray)

            color = (255, 0, 0)
            thickness = 2
            end_cord_x = x + w
            end_cord_y = y + h
            cv2.rectangle(frame, (x, y), (end_cord_x, end_cord_y), color, thickness)

        cv2.imshow('frame', frame)

        if cv2.waitKey(20) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    
    
def train_data():
    BASE_DIR = os.getcwd()
    image_dir = os.path.join(BASE_DIR, "images")


    face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
    recognizer = cv2.face.LBPHFaceRecognizer_create()

    current_id = 0
    label_ids = {}
    y_labels = []
    x_train = []

    for root, dirs, files in os.walk(image_dir):
        for file in files:
            if file.endswith("png") or file.endswith("jpg"):
                path = os.path.join(root, file)
                label = os.path.basename(root).replace(" ", "-").lower()
                print(label, path)
                if not label in label_ids:
                    label_ids[label]= current_id
                    current_id +=1
                id_= label_ids[label]
                print(label_ids)
            
                pil_image = Image.open(path).convert("L")
                size = (550,550)
                final_image = pil_image.resize(size,Image.Resampling.LANCZOS)
                image_array = np.array(final_image, "uint8")
                #print(image_array)
                faces = face_cascade.detectMultiScale(image_array, scaleFactor=1.5, minNeighbors=5)
            
            
                for(x,y,w,h) in faces:
                    roi= image_array[y:y+h, x:x+w]
                    x_train.append(roi)
                    y_labels.append(id_)

            
            
    with open("pickles/face-labels.pickle", 'wb') as f:
	    pickle.dump(label_ids, f)

    recognizer.train(x_train, np.array(y_labels))
    recognizer.save("recognizers/face-trainner.yml")
    

# Set up the tkinter app window
app.title('Select Images')
app.geometry('400x250')
app.resizable(False, False)
screen_width = app.winfo_screenwidth()
screen_height = app.winfo_screenheight()
x = int((screen_width - app.winfo_reqwidth()) / 2)
y = int((screen_height - app.winfo_reqheight()) / 2)
app.geometry(f'+{x}+{y}')
label = tk.Label(app, text='Enter the name of the user:')
label.pack()
folder_name_entry = tk.Entry(app)
folder_name_entry.pack()
button_frame = tk.Frame(app)
button_frame.pack()
select_button = tk.Button(button_frame, text='Select', command=select_folder)
select_button.pack(side=tk.LEFT, padx=5)
train_button = tk.Button(button_frame, text='train', command=train_data)
train_button.pack(side=tk.LEFT, padx=5) 
restart_button = tk.Button(button_frame, text='Restart', command=restart_app)
restart_button.pack(side=tk.LEFT, padx=5)
delete_button = tk.Button(button_frame, text='Delete User', command=delete_user) 
delete_button.pack(side=tk.LEFT, padx=5)
exit_button = tk.Button(button_frame, text='Exit', command=exit_app)
exit_button.pack(side=tk.LEFT, padx=5)
open_button = tk.Button(button_frame, text='Open Camera', command=open_cam)
open_button.pack(side=tk.LEFT, padx=5) 

app.mainloop()